In [3]:
#-----------------------------------#
# Target: for preparing the data before runing FOST forecsating model
# FOST only receives .xlsx or pandas.DataFrame
# DATA: IMFS and smapi within Huai river Plain
# This practice is to remove all the invalid grids (for reducing computation) 
# author: X.W.
# 6-23-2022
#-----------------------------------#
import xarray as xr
import pandas as pd
#import geopandas as gpd
import numpy as np
import os
import glob

In [4]:
# 读取文件名的函数
def read_file(path, file_type):  # path 是指需要提取的文件夹所在路径;file_type是指文件的拓展名;txt格式的文件则输入‘.txt’
    filenames = os.listdir(path)
    filenames1 = []
    for filename in filenames:
        if os.path.splitext(filename)[1] == file_type:
            filenames1.append(filename)
    return filenames1

In [5]:
os.chdir(r'E:\Data_inputs\masked_data_HRB')  # 更改工作路径
curr_path = os.getcwd()  # 获取当前路径
files = read_file(curr_path, '.nc')  # 利用上述函数获取当前文件夹内的.nc文件名
# 最后一个文件是smapi，注意区分
files

['imfs0_HRB_from_1980_to_2021.nc',
 'imfs1_HRB_from_1980_to_2021.nc',
 'imfs2_HRB_from_1980_to_2021.nc',
 'imfs3_HRB_from_1980_to_2021.nc',
 'imfs4_HRB_from_1980_to_2021.nc',
 'imfs5_HRB_from_1980_to_2021.nc',
 'imfs6_HRB_from_1980_to_2021.nc',
 'smapi_HRB_from_1980_to_2021.nc']

In [8]:
def save_for_fost(data,k):
    if k == 7:
        dt = data['smapi'].values
    else: 
        dt = data['imf'].values
    df = pd.DataFrame(dt.reshape((dt.shape[0],-1)))
    id_nan = df[df.columns[df.isnull().any()]].columns
    # 获得均为nan的列，df_new = df.iloc[:,id_nan.values]
    df = df.dropna(axis=1, how='all')
    t_break = 2042
    df_train = df.loc[:t_break,:]
    #df_val =  df.loc[t_break:,:]
    df_date = pd.DataFrame()
    df_date['Date'] = data.time[:t_break+1]
    #[pd.datetime(year=1979,month=12,day=31)+pd.Timedelta(weeks=i) for i in range(df_train.shape[0])] 
    n = df_train.shape[1]
    df_date = pd.concat([df_date] * n, axis=1, ignore_index=True)
    df_train = pd.melt(df_train)
    df_date = pd.melt(df_date)
    df_train['Date'] = df_date['value']
    df_train.rename(columns = {'variable':'Node',
                               'value':'TARGET'}, inplace = True)
    #df_train.to_pickle('E:\\Data_inputs\\fost_inputs_train\\imfs_'+str(k)+'_HRB_pd_melt.pkl')
    df_train.to_csv('E:\\Data_inputs\\fost_inputs_train\\imfs_'+str(k)+'_HRB_pd_melt.csv')
#'E:\\Data_inputs\\masked_data_HRB\\imfs'+str(ii)+'_HRB_from_1980_to_2021.nc'

In [9]:
# note: imfs_7_HRB_pd_melt.pkl represants the SMAPI_pandas

for ii in range(len(files)):
    med_data = xr.open_dataset(files[ii])
    save_for_fost(med_data,ii)
    print(ii)

0
1
2
3
4
5
6
7


In [17]:
def save_for_fost_val(data,k):
    if k == 7:
        dt = data['smapi'].values
    else: 
        dt = data['imf'].values
    df = pd.DataFrame(dt.reshape((dt.shape[0],-1)))
    id_nan = df[df.columns[df.isnull().any()]].columns
    # 获得均为nan的列，df_new = df.iloc[:,id_nan.values]
    df = df.dropna(axis=1, how='all')
    t1 = 1800
    t2 = 2052
    df_val = df.loc[t1:t2,:]
    #df_val =  df.loc[t_break:,:]
    df_date = pd.DataFrame()
    df_date['Date'] = data.time[t1:t2+1]
    n = df_val.shape[1]
    df_date = pd.concat([df_date] * n, axis=1, ignore_index=True)
    df_val = pd.melt(df_val)
    df_date = pd.melt(df_date)
    df_val['Date'] = df_date['value']
    df_val.rename(columns = {'variable':'Node',
                               'value':'TARGET'}, inplace = True)
    #df_train.to_pickle('E:\\Data_inputs\\fost_inputs_train\\imfs_'+str(k)+'_HRB_pd_melt.pkl')
    df_val.to_csv('E:\\Data_inputs\\fost_inputs_val\\imfs_'+str(k)+'_HRB_pd_melt_VAL.csv')
#'E:\\Data_inputs\\masked_data_HRB\\imfs'+str(ii)+'_HRB_from_1980_to_2021.nc'

In [18]:
for ii in range(len(files)):
    med_data = xr.open_dataset(files[ii])
    save_for_fost_val(med_data,ii)
    print(ii)

0
1
2
3
4
5
6
7


In [4]:
# select the first file as an example
data = xr.open_dataset(files[0])
dt = data['imf'].values
df = pd.DataFrame(dt.reshape((dt.shape[0],-1)))
id_nan = df[df.columns[df.isnull().any()]].columns
# 获得均为nan的列，df_new = df.iloc[:,id_nan.values]
df = df.dropna(axis=1, how='all')
#df.columns[0]

In [21]:
# 从第2043周开始进入2019.3，意味着进入干旱多发期 
data.time[2042]

<xarray.DataArray 'time' ()>
array('2019-02-24', dtype='<U10')
Coordinates:
    time            <U10 '2019-02-24'
    reference_time  datetime64[ns] 2022-06-23T20:50:39.019503

In [24]:
t_break = 2042
df_train = df.loc[:t_break,:]
#df_val =  df.loc[t_break:,:]
df_date = pd.DataFrame()
df_date['Date'] = data.time[:t_break+1]
#[pd.datetime(year=1979,month=12,day=31)+pd.Timedelta(weeks=i) for i in range(df_train.shape[0])] 
n = df_train.shape[1]
df_date = pd.concat([df_date] * n, axis=1, ignore_index=True)

In [25]:
df_train = pd.melt(df_train)
df_date = pd.melt(df_date)
df_train['Date'] = df_date['value']
df_train.rename(columns = {'variable':'Node',
                           'value':'TARGET'}, inplace = True)
df_train

,Node,TARGET,Date
0,689,-0.298945,1980-01-06
1,689,0.278460,1980-01-13
2,689,-0.226318,1980-01-20
3,689,0.160598,1980-01-27
4,689,-0.012513,1980-02-03
...,...,...,...
5305666,6080,-0.016630,2019-01-27
5305667,6080,0.029337,2019-02-03
5305668,6080,-0.017165,2019-02-10
5305669,6080,-0.065834,2019-02-17


In [26]:
df_train.to_pickle(r'E:\Data_inputs\fost_inputs_train\imf1_pd_melt.pkl')

In [27]:
dt_test = pd.read_pickle(r'E:\Data_inputs\fost_inputs_train\imf1_pd_melt.pkl')

In [28]:
dt_test

,Node,TARGET,Date
0,689,-0.298945,1980-01-06
1,689,0.278460,1980-01-13
2,689,-0.226318,1980-01-20
3,689,0.160598,1980-01-27
4,689,-0.012513,1980-02-03
...,...,...,...
5305666,6080,-0.016630,2019-01-27
5305667,6080,0.029337,2019-02-03
5305668,6080,-0.017165,2019-02-10
5305669,6080,-0.065834,2019-02-17
